In [ ]:
!pip install transformers datasets accelerate evaluate torch torchvision
!pip install --upgrade transformers datasets
!pip install requests scikit-learn

In [ ]:
import os
from datasets import load_dataset
from transformers import ViTImageProcessor

# 전체 데이터셋 로드
dataset = load_dataset("food101", split="train")
eval_dataset = load_dataset("food101", split="validation")

# ViT 모델에 맞는 전처리
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

def preprocess_images(examples):
    images = [image.convert("RGB") for image in examples["image"]]
    examples["pixel_values"] = processor(images=images, return_tensors="pt").pixel_values
    return examples

# 데이터셋에 전처리 함수 적용
processed_dataset = dataset.map(preprocess_images, batched=True, num_proc=os.cpu_count())
processed_eval_dataset = eval_dataset.map(preprocess_images, batched=True, num_proc=os.cpu_count())

# 필요 없는 원본 이미지 컬럼 제거 및 텐서 형식 설정
processed_dataset.set_format(type="torch", columns=["pixel_values", "label"])
processed_eval_dataset.set_format(type="torch", columns=["pixel_values", "label"])

# 클래스 이름 가져오기
labels = processed_dataset.features["label"].names
id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}

ValueError: Invalid pattern: '**' can only be an entire path component

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import ViTForImageClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
import torch
import os

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

training_args = TrainingArguments(
    output_dir="./vit-food101-finetuned",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    logging_steps=10,

    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    eval_dataset=processed_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.save_model("./vit-food101-finetuned")
processor.save_pretrained("./vit-food101-finetuned")
print("저장 완료")

ModuleNotFoundError: No module named 'evaluate'

In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import os
import shutil

# 학습된 모델과 프로세서 로드
processor = ViTImageProcessor.from_pretrained("./vit-food101-finetuned")
model = ViTForImageClassification.from_pretrained("./vit-food101-finetuned")

print("학습된 모델과 프로세서 로드 완료")

학습된 모델과 프로세서 로드 완료


In [ ]:
import torch

def classify_and_organize_food_images(image_folder_path, output_base_path):

    # 출력 기본 경로가 없으면 생성
    if not os.path.exists(output_base_path):
        os.makedirs(output_base_path)
        print(f"출력 폴더 생성: {output_base_path}")

    # 지정된 이미지 폴더 내의 모든 파일 순회
    for filename in os.listdir(image_folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            image_full_path = os.path.join(image_folder_path, filename)
            print(f"이미지 처리 중: {image_full_path}")

            try:
                # 이미지 로드 및 RGB로 변환 (일부 이미지 형식은 RGB가 아닐 수 있음)
                image = Image.open(image_full_path).convert("RGB")

                # 이미지 전처리
                inputs = processor(images=image, return_tensors="pt")

                # GPU로 이동
                if torch.cuda.is_available():
                    inputs = {k: v.to("cuda") for k, v in inputs.items()}
                    model.to("cuda")

                outputs = model(**inputs) # 모델에 이미지 입력하여 예측 수행
                logits = outputs.logits

                # 가장 높은 점수를 가진 클래스의 인덱스 추출
                predicted_class_idx = logits.argmax(-1).item()
                predicted_label = model.config.id2label[predicted_class_idx]

                # 예측된 레이블(음식 종류) 이름으로 폴더 경로 생성
                output_food_folder = os.path.join(output_base_path, predicted_label)
                if not os.path.exists(output_food_folder):
                    os.makedirs(output_food_folder)

                # 원본 이미지를 예측된 음식 종류 폴더로 복사
                shutil.copy(image_full_path, os.path.join(output_food_folder, filename))
                print("분류 및 복사 완료")

            except Exception as e:
                print(f"오류 발생 ({filename}): {e}")


# 예시 이미지들이 있는 폴더 경로
my_image_folder = "/content/drive/MyDrive/ColabNotebooks/MyFoodPhotos_Example"

# 분류된 음식 사진들이 저장될 폴더 경로
output_food_classified_folder = "/content/drive/MyDrive/FoodOrganizedByDish_Output"

# 이미지 분류 및 정리 시작
classify_and_organize_food_images(my_image_folder, output_food_classified_folder)
print("정리 완료")


def export_classified_images_to_csv(output_base_path, export_path_csv="classified_results.csv", export_path_xlsx=None):
    records = []

    for label in os.listdir(output_base_path):
        class_folder = os.path.join(output_base_path, label)
        if os.path.isdir(class_folder):
            for image_file in os.listdir(class_folder):
                if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    full_path = os.path.join(class_folder, image_file)
                    records.append({
                        "image_filename": image_file,
                        "predicted_label": label,
                        "full_path": full_path
                    })
import pandas as pd
import os

def export_classified_images_to_csv(output_base_path, export_path_csv="classified_results.csv", export_path_xlsx=None):
    records = []

    for label in os.listdir(output_base_path):
        class_folder = os.path.join(output_base_path, label)
        if os.path.isdir(class_folder):
            for image_file in os.listdir(class_folder):
                if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    full_path = os.path.join(class_folder, image_file)
                    records.append({
                        "image_filename": image_file,
                        "predicted_label": label,
                        "full_path": full_path
                    })

    df = pd.DataFrame(records)
    df.to_csv(export_path_csv, index=False)


    if export_path_xlsx:
        df.to_excel(export_path_xlsx, index=False)
        print(f"XLSX 파일 저장 완료")

export_classified_images_to_csv(
    output_base_path=output_food_classified_folder,
    export_path_csv="/content/drive/MyDrive/food_classified_summary.csv",
    export_path_xlsx="/content/drive/MyDrive/food_classified_summary.xlsx"
)


음식 이미지 분류 및 정리 시작
출력 폴더 생성: /content/drive/MyDrive/FoodOrganizedByDish_Output
이미지 처리 중: /content/drive/MyDrive/ColabNotebooks/MyFoodPhotos_Example/MyFoodExample (347).JPG
  -> 'MyFoodExample (347).JPG'를 'ceviche' 폴더로 분류 및 복사 완료.
이미지 처리 중: /content/drive/MyDrive/ColabNotebooks/MyFoodPhotos_Example/MyFoodExample (332).JPG
  -> 'MyFoodExample (332).JPG'를 'pulled_pork_sandwich' 폴더로 분류 및 복사 완료.
이미지 처리 중: /content/drive/MyDrive/ColabNotebooks/MyFoodPhotos_Example/MyFoodExample (330).JPG
  -> 'MyFoodExample (330).JPG'를 'deviled_eggs' 폴더로 분류 및 복사 완료.
이미지 처리 중: /content/drive/MyDrive/ColabNotebooks/MyFoodPhotos_Example/MyFoodExample (326).JPG
  -> 'MyFoodExample (326).JPG'를 'pulled_pork_sandwich' 폴더로 분류 및 복사 완료.
이미지 처리 중: /content/drive/MyDrive/ColabNotebooks/MyFoodPhotos_Example/MyFoodExample (331).JPG
  -> 'MyFoodExample (331).JPG'를 'beignets' 폴더로 분류 및 복사 완료.
이미지 처리 중: /content/drive/MyDrive/ColabNotebooks/MyFoodPhotos_Example/MyFoodExample (322).JPG
  -> 'MyFoodExample (322).JPG'를 'chicken_

In [ ]:
from PIL import Image, ExifTags
from PIL.ExifTags import TAGS, GPSTAGS
import shutil
from sklearn.cluster import DBSCAN
import requests
import json
from collections import defaultdict


def get_exif_data(image_path):
    exif_data = {}
    try:
        with Image.open(image_path) as img:
            img.verify()
            info = img._getexif()
            if info:
                for tag, value in info.items():
                    decoded = TAGS.get(tag, tag)
                    if decoded == "GPSInfo":
                        gps_data = {}
                        for t in value:
                            sub_decoded = GPSTAGS.get(t, t)
                            gps_data[sub_decoded] = value[t]
                        exif_data[decoded] = gps_data
                    else:
                        exif_data[decoded] = value
    except (IOError, AttributeError, KeyError, ValueError) as e:
        pass
    return exif_data

def convert_dms_to_degrees(dms):
    """도, 분, 초(DMS) 형식을 십진수 도(Degrees)로 변환합니다."""
    degrees = float(dms[0])
    minutes = float(dms[1]) / 60.0

    seconds_val = dms[2]
    if isinstance(seconds_val, tuple) and len(seconds_val) == 2:
        seconds = float(seconds_val[0]) / seconds_val[1] / 3600.0
    else:
        seconds = float(seconds_val) / 3600.0

    return degrees + minutes + seconds

def get_gps_coordinates(exif_data):
    lat, lon = None, None
    if "GPSInfo" in exif_data:
        gps_info = exif_data["GPSInfo"]
        gps_latitude = gps_info.get("GPSLatitude")
        gps_latitude_ref = gps_info.get("GPSLatitudeRef")
        gps_longitude = gps_info.get("GPSLongitude")
        gps_longitude_ref = gps_info.get("GPSLongitudeRef")

        if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
            try:
                lat = convert_dms_to_degrees(gps_latitude)
                if gps_latitude_ref == "S":
                    lat = -lat

                lon = convert_dms_to_degrees(gps_longitude)
                if gps_longitude_ref == "W":
                    lon = -lon
            except Exception as e:
                lat, lon = None, None
    return lat, lon


def get_location_name_from_coords(lat, lon):
    url = f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json"
    headers = {'User-Agent': 'FoodPhotoOrganizerApp/1.0 (your_email@example.com)'}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()

        if 'display_name' in data:
            address = data.get('address', {})
            name = address.get('amenity') or \
                   address.get('building') or \
                   address.get('road') or \
                   address.get('town') or \
                   address.get('city') or \
                   data['display_name'].split(',')[0].strip()

            safe_name = "".join(c for c in name if c.isalnum() or c in (' ', '_', '-')).strip()
            safe_name = safe_name.replace(" ", "_")
            if not safe_name:
                return f"Location_{lat:.4f}_{lon:.4f}"
            return safe_name

        return f"Unknown_Loc_{lat:.4f}_{lon:.4f}"
    except requests.exceptions.RequestException as e:
        return f"API_Error_Loc_{lat:.4f}_{lon:.4f}"
    except json.JSONDecodeError as e:
        return f"Parse_Error_Loc_{lat:.4f}_{lon:.4f}"
    except Exception as e:
        return f"Unhandled_Error_Loc_{lat:.4f}_{lon:.4f}"


def organize_images_by_location(food_classified_base_path, output_location_base_path, eps=0.0001, min_samples=2):

    if not os.path.exists(output_location_base_path):
        os.makedirs(output_location_base_path)
        print(f"최종 출력 폴더 생성: {output_location_base_path}")

    all_images_with_gps = []
    image_paths_without_gps = []

    # 모든 음식 폴더를 순회하며 이미지와 GPS 정보 수집
    processed_count = 0
    for food_type_folder in os.listdir(food_classified_base_path):
        food_type_path = os.path.join(food_classified_base_path, food_type_folder)
        if os.path.isdir(food_type_path):
            for filename in os.listdir(food_type_path):
                image_path = os.path.join(food_type_path, filename)
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    exif_data = get_exif_data(image_path)
                    lat, lon = get_gps_coordinates(exif_data)
                    if lat is not None and lon is not None:
                        all_images_with_gps.append((image_path, lat, lon))
                    else:
                        image_paths_without_gps.append(image_path)
                    processed_count += 1
                    if processed_count % 100 == 0:
                        print("정보 수집 완료")


    # GPS 정보가 있는 이미지가 있을 경우에만 클러스터링 및 위치 기반 분류 수행
    if all_images_with_gps:
        coordinates = np.array([[lat, lon] for _, lat, lon in all_images_with_gps])

        db = DBSCAN(eps=eps, min_samples=min_samples).fit(coordinates)
        labels = db.labels_

        location_clusters = defaultdict(list)
        cluster_center_names = {}

        for i, (image_path, lat, lon) in enumerate(all_images_with_gps):
            cluster_label = labels[i]
            if cluster_label == -1:
                location_clusters["Unclustered_Location"].append(image_path)
            else:
                if cluster_label not in cluster_center_names:
                    cluster_coords_in_group = [coord for j, coord in enumerate(coordinates) if labels[j] == cluster_label]
                    cluster_center_lat = np.mean([coord[0] for coord in cluster_coords_in_group])
                    cluster_center_lon = np.mean([coord[1] for coord in cluster_coords_in_group])

                    location_name = get_location_name_from_coords(cluster_center_lat, cluster_center_lon)
                    cluster_center_names[cluster_label] = location_name

                location_clusters[cluster_center_names[cluster_label]].append(image_path)

        for location_name, img_paths in location_clusters.items():
            output_location_folder = os.path.join(output_location_base_path, location_name)
            if not os.path.exists(output_location_folder):
                os.makedirs(output_location_folder)
                print(f"  위치 폴더 생성: {location_name}")

            for img_path in img_paths:
                shutil.copy(img_path, os.path.join(output_location_folder, os.path.basename(img_path)))
    else:
        print("실패")


    # GPS 정보가 없는 이미지 처리
    if image_paths_without_gps:
        no_gps_folder = os.path.join(output_location_base_path, "No_GPS_Info")
        if not os.path.exists(no_gps_folder):
            os.makedirs(no_gps_folder)
            print("위치 없는 이미지 폴더 생성: {no_gps_folder}")
        for img_path in image_paths_without_gps:
            shutil.copy(img_path, os.path.join(no_gps_folder, os.path.basename(img_path)))
            # print(f"  -> '{os.path.basename(img_path)}'를 'No_GPS_Info' 폴더로 복사 완료.")

    print("정리 완료")

food_classified_base_folder = "/content/drive/MyDrive/FoodOrganizedByDish_Output".
final_location_organized_folder = "/content/drive/MyDrive/FoodOrganizedByLocation_Output"

organize_images_by_location(food_classified_base_folder, final_location_organized_folder, eps=0.0005, min_samples=3)
print("위치 기반 이미지 분류 및 정리 완료")


위치 기반 이미지 분류 및 정리 시작
최종 출력 폴더 생성: /content/drive/MyDrive/FoodOrganizedByLocation_Output
GPS 정보 수집 중...
  100개 이미지 GPS 정보 수집 완료...
  200개 이미지 GPS 정보 수집 완료...
  300개 이미지 GPS 정보 수집 완료...
총 279개의 GPS 정보 있는 이미지 클러스터링 중...
  위치 폴더 생성: 60계_치킨_서울한양대점
  위치 폴더 생성: Unclustered_Location
  위치 폴더 생성: 죽이야기파주금촌점
  위치 폴더 생성: 커피팜
  위치 폴더 생성: 신동면
  위치 폴더 생성: 쌩판
  위치 폴더 생성: 가나무로
  위치 폴더 생성: 파주시
  위치 폴더 생성: 월드컵북로48길
  위치 폴더 생성: 통일로771번길
  위치 폴더 생성: 김종용누룽지통닭
  위치 폴더 생성: 통일로770번길
  위치 폴더 생성: 금빛로
  위치 폴더 생성: 경의로
  위치 폴더 생성: 압구정화로구이한양대점
  위치 폴더 생성: 삼성빌라트_A동
  위치 폴더 생성: 동남로13길
  위치 폴더 생성: 북한강변로
  위치 폴더 생성: 번영로
  위치 폴더 생성: 아름다운교회
  위치 폴더 생성: 한양대학교_서울캠퍼스
  위치 폴더 생성: 여의대로
  위치 폴더 생성: 세종대로2길
  위치 폴더 생성: 장충단로
  위치 폴더 생성: 통일로754번길
  위치 폴더 생성: 애니골길
GPS 정보 없는 이미지 폴더 생성: /content/drive/MyDrive/FoodOrganizedByLocation_Output/No_GPS_Info

위치 기반 이미지 분류 및 정리 완료
